In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from sympy import *
from scipy.integrate import odeint

In the following cell I'm calculating $Y_{eq}$ for a W and Z bosons. \
I'm using boltzmann equation to calculate the number density $n$. 
$$n = \tfrac{g}{(2 \pi)^3} \int_{0}^{\inf} \tfrac{p^2}{exp(\sqrt{p^2+m^2}/T)-1} \,dx $$

$$Y_{eq} = n/T^3$$

In [4]:
#This function calculates the equilibrium abundance of a boson.
# T is temperature in Gev, M is mass of particle in Gev. 
#for w and Z bosons, M = 80 Gev. 
def Equilibrium_abundance(T,m,g):
    result = quad(lambda p: p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1),0, np.inf)
    Value = ((9/(2*np.pi**2))*result[0])/T**3
    return Value

Equilibrium_abundance(80,80,9)



/tmp/ipykernel_10121/1038210693.py:5: RuntimeWarning: overflow encountered in exp
  result = quad(lambda p: p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1),0, np.inf)


0.8106839724807485

This is where I make a list of all $Y_{eq}$ values. I wanted to make an array, but it was taking me longer than expected to figure it out. 


In [5]:
#make Yeq list
Ts= np.logspace(2, 0, 100)
def Yeq_list(Tlist):
    Yeq = []
    for T in Tlist:
        Yeq.append(Equilibrium_abundance(T, 80,9)**2)
    return Yeq

#Yeq_list(Ts)
#print(Yeq)
#for i in Yeq: 
#    if i < 0: 
#        print(i)

This is where I calculate $\lambda$ eq 6.13 in Huterer

In [6]:
#This function calculates lambda in the expression for particle abundance
#where cross_section is particle cross_section, 
#m is mass, 
#gstar, is effective number of relativistic degress of freedom
#degrees of freedom is g 
def Lambda(cross_section,m,gstar,g):
    #planck mass in Gev 
    mpl = 1.220890* 10**19
    Hubble_constant = 1.66*g**(1/2)*(m**2/mpl)
    value = 2*(3.14)**2/45*gstar*((m**3*cross_section)/(Hubble_constant))
    return value
    
Lambda(10**5,80,86.25,9)

7.412642624149934e+26

This is the model I came up with by rewriting $dY/dx$ as $dlnY/lnx$

$$\tfrac{dLnY}{dLnx} = - \tfrac{\lambda}{x} \left[Y- \tfrac{Y_{eq}^2}{Y} \right]$$

In [7]:
def model(y,x):
    L = 7.412642624149934e+26
    dydx = -L/x * (y-Yeq_list(Ts)/y)
    return dydx


Log of the model I mentioned above.

In [25]:
def model2(y,x):
    L = 7.412642624149934e+26
    dydx = -L/np.log(x) * (np.log(y)-Yeq_list(Ts)[1]/np.log(y))
    return dydx


This is the equation given in the book that the question says I need to solve. eq 6.14 

In [30]:
def model3(y,x):
    L = 7.412642624149934e+26
    dydx = -L/x**2 * (y**2-Yeq_list(Ts)**2)
    return dydx

In [ ]:
plt.loglog(x,y)
plt.xlabel('time')
plt.ylabel('y(x)')
plt.show()

when I try to solve the ode of any of the 3 models, I get an error that relates to my model diverging. I do not know what to do about it. 

In [8]:
# initial condition
y0 = np.ones(100)

# time points
x = np.log(80)*np.ones(100)/np.logspace(2,0,100)
0
# solve ODE
y = odeint(model,y0,x)

print(y)

/tmp/ipykernel_10121/1038210693.py:5: RuntimeWarning: overflow encountered in exp
  result = quad(lambda p: p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1),0, np.inf)


 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.1443253329003D-31
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.1443253329003D-31
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.2997739074419D-30
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.2997739074419D-30
 lsoda--  warning..internal t (=r1) 

/home/ebtihal/miniconda3/lib/python3.9/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
